In [14]:
import math
from Bio.PDB import PDBParser
from sph_saff import calc_points  # ton script Saff/Kuiljaars
from anti_lib import Vec  # si nécessaire pour les points
from types import SimpleNamespace

args = SimpleNamespace(number_points=92, angle=None, exclude_poles=False, outfile=None)
points_unitaires = calc_points(args)

# --- paramètres ---
R_sonde = 1.4  # rayon de l'eau en Å
VDW_radii = {'C': 1.7, 'N': 1.55, 'O': 1.52, 'S': 1.8, 'H': 1.2}  # ex

# --- charger la structure ---
parser = PDBParser(QUIET=True)
structure = parser.get_structure("test", "1AKI.pdb")

# prendre un atome au hasard pour test
atom = list(structure.get_atoms())[0]

# rayon sphère = VdW + rayon sonde
rayon_sphere = VDW_radii.get(atom.element, 1.5) + R_sonde

# créer la sphère de points
class Args:
    number_points = 92
    angle = None

args = SimpleNamespace(number_points=92, angle=None, exclude_poles=False, outfile=None)
points_unitaires = calc_points(args)
points_atom = [(atom.coord[0] + p.x*rayon_sphere,
                atom.coord[1] + p.y*rayon_sphere,
                atom.coord[2] + p.z*rayon_sphere)
               for p in points_unitaires]

# --- vérifier l'accessibilité des points ---
accessible_points = 0
all_atoms = list(structure.get_atoms())
for px, py, pz in points_atom:
    is_accessible = True
    for other in all_atoms:
        if other == atom:
            continue
        r = VDW_radii.get(other.element, 1.5) + R_sonde
        dx, dy, dz = px - other.coord[0], py - other.coord[1], pz - other.coord[2]
        if dx*dx + dy*dy + dz*dz < r*r:
            is_accessible = False
            break
    if is_accessible:
        accessible_points += 1

# --- calcul SASA ---
surface_point = 4 * math.pi * rayon_sphere**2 / Args.number_points
SASA_atom = accessible_points * surface_point

print(f"Atome: {atom.get_name()}, SASA = {SASA_atom:.2f} Å²")


TypeError: argument of type 'types.SimpleNamespace' is not iterable